In [1]:
import numpy as np
import scipy.io
from midiutil import MIDIFile
from utils import *

In [2]:
key_time = 50e-3*0.5
Fs = 48000

## picked keys
keys = scipy.io.loadmat('MATLAB_data/23Feb_jc_key_2octave.mat')['key']

##freq of notes on piano
notes_list = scipy.io.loadmat('notes_112.mat')['notes'] 
#notes_list = scipy.io.loadmat('notes_100.mat')['notes'] 
# notes_list = scipy.io.loadmat('notes_88.mat')['notes']

##freq of t-f components
freq_list = scipy.io.loadmat('freq_480_two_octave_down.mat')['f_cropped'] 
#freq_list = scipy.io.loadmat('freq_480_octave_down.mat')['f_cropped'] 
# freq_list = scipy.io.loadmat('freq_480.mat')['f_cropped']
# freq_list = scipy.io.loadmat('freq_441.mat')['f_cropped']


notes_list = notes_list.reshape(np.size(notes_list))
freq_list = freq_list.reshape(np.size(freq_list))

note_num = 112 #88+12, 12 from upper octave

intensity_map = scipy.io.loadmat('MATLAB_data/23Feb_jc_int_2octave.mat')['filtered_int_db']

In [4]:
def intensity2sign(intensity):
    thresholds = [35,55,75]
    if intensity < thresholds[0]:
        return 'p'
    elif intensity < thresholds[1]:
        return 'mp'
    elif intensity < thresholds[2]:
        return 'mf'
    else:
        return 'f'

In [5]:
def picknotes(cur_key, cur_freq, cur_int, notes, notes_list):
    j = 0
    while j < len(cur_key):
        if cur_key[j] == 0:
            j += 1
            continue
        # if cur_key[j] > 88:
        #     pitch = cur_key[j] - 12
        # else:
        #     pitch = cur_key[j]
        pitch = cur_key[j]
        while pitch > 88:
            pitch -= 12
        
        start = j
        ori_int = cur_int[j]
        while cur_key[j] != 0 and j < len(cur_key)-1:
            j += 1
            ori_int = max(ori_int,cur_int[j])
        end = j - 1
        if end - start < 2: 
            end = start + 2 #all to 100ms
     
        intensity = get_intensity(cur_key[j-1],cur_freq,notes_list) * ori_int
        #intensity = ori_int
        #intensity = get_intensity(cur_key[j-1],cur_freq,notes_list) * 100
        intensity_sign = intensity2sign(intensity)
        #intensity_sign = intensity2sign(intensity)
        notes.append(Note(pitch, start, end, intensity, intensity_sign))
        j += 1
    return notes

In [6]:
notes = []
for i in range(np.shape(keys)[0]):
    cur_key = keys[i]
    cur_int = intensity_map[i]
    cur_freq = freq_list[i]
    notes = picknotes(cur_key, cur_freq, cur_int, notes, notes_list)

print(len(notes))

4131


In [7]:
notes = sorted(notes, key = lambda note: note.start)
new_notes = remove_repetitive(notes,note_num)
# pprint(new_notes)
print(len(new_notes))

649


In [8]:
nnew_notes = multichannel(new_notes,note_num)

In [9]:
mf = MIDIFile(1)     # only 1 track
track = 0   # the only track
time = 0    # start at the beginning
mf.addTrackName(track, time, "Sample Track")
mf.addTempo(track, time, 1200)

In [10]:
for note in nnew_notes:
    v = int(20+note.sign()*20-40)
    #v = int(50+note.sign()*12.5)
    #v = int(note.intensity)
    mf.addNote(track,note.channel,note.pitch+20, note.start, note.end-note.start+1,v)

In [11]:
with open("midifile/23Feb_jc_5.mid", 'wb') as outf:
   mf.writeFile(outf)

In [ ]:
plot_notes(new_notes)

In [ ]:
# def picknotes_old(cur_key,cur_freq):
#     j = 0
#     while j < len(cur_key):
#         if cur_key[j] != 0:
            
#             d = {}
#             d['note'] = cur_key[j]
#             d['start'] = j
#             while cur_key[j] != 0 and j < len(cur_key)-1:
#                 j += 1
#             d['end'] = j - 1
#             intensity = get_intensity(cur_key[j-1],cur_freq,notes_list[cur_key[j-1]-1])
#             d['intensity'] = intensity
#             notes.append(d)
#         j += 1